In [1]:
from dotenv import load_dotenv
load_dotenv('../.env')

True

In [2]:
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

In [3]:
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
]

In [4]:
# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: 2+2
AI: 4
Human: 2+3
AI: 5


In [5]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [6]:
final_prompt 

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a wondrous wizard of math.')), FewShotChatMessagePromptTemplate(examples=[{'input': '2+2', 'output': '4'}, {'input': '2+3', 'output': '5'}], example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], template='{output}'))])), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [11]:
# print the content of final_prompt as a string
print(final_prompt.format(input='test input'))

System: You are a wondrous wizard of math.
Human: 2+2
AI: 4
Human: 2+3
AI: 5
Human: test input


In [14]:
from langchain_community.chat_models import ChatOpenAI

chain = final_prompt | ChatOpenAI(temperature=0.0)

chain.invoke({"input": "What's the square of a triangle?"})

c:\Users\Asus\anaconda3\envs\dev\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


AIMessage(content='A square is a type of quadrilateral with four equal sides and four right angles. A triangle is a polygon with three sides. Therefore, a square of a triangle is not a valid mathematical concept.')

In [15]:
chain.invoke({"input": "5**3"})

AIMessage(content='125')

In [16]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [17]:
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "2+4", "output": "6"},
    {"input": "What did the cow say to the moon?", "output": "nothing at all"},
    {
        "input": "Write me a poem about the moon",
        "output": "One for the moon, and one for me, who are we to talk about the moon?",
    },
]

to_vectorize = [" ".join(example.values()) for example in examples]
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

In [18]:
to_vectorize

['2+2 4',
 '2+3 5',
 '2+4 6',
 'What did the cow say to the moon? nothing at all',
 'Write me a poem about the moon One for the moon, and one for me, who are we to talk about the moon?']

In [19]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

# The prompt template will load examples by passing the input do the `select_examples` method
example_selector.select_examples({"input": "horse"})

[{'input': 'What did the cow say to the moon?', 'output': 'nothing at all'},
 {'input': '2+4', 'output': '6'}]

In [20]:
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

# Define the few-shot prompt.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    input_variables=["input"],
    example_selector=example_selector,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

In [21]:
print(few_shot_prompt.format(input="What's 3+3?"))

Human: 2+3
AI: 5
Human: 2+2
AI: 4


In [24]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [25]:
final_prompt.format(input="What's 3+3?")

"System: You are a wondrous wizard of math.\nHuman: 2+3\nAI: 5\nHuman: 2+2\nAI: 4\nHuman: What's 3+3?"

In [27]:
from langchain_community.chat_models import ChatOpenAI

chain = final_prompt | ChatOpenAI(temperature=0.0)

chain.invoke({"input": "What's 3+3?"})

AIMessage(content='6')